<a href="https://colab.research.google.com/github/hyejwon/1_ne/blob/main/Keras_Tuner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 개요
Keras Tuner는 TensorFlow 프로그램에 대한 최적의 하이퍼파라미터 세트를 선택하는 데 도움을 주는 라이브러리입니다. 머신러닝(ML) 애플리케이션에 대한 올바른 하이퍼파라미터 세트를 선택하는 과정을 하이퍼파라미터 조정 또는 하이퍼튜닝이라고 합니다.

하이퍼파라미터는 훈련 프로세스 및 ML 모델의 토폴로지를 제어하는 변수입니다. 이러한 변수는 훈련 과정에서 일정하게 유지되며 ML 프로그램의 성능에 직접적으로 영향을 미칩니다. 하이퍼파라미터에는 두 가지 유형이 있습니다.

1. 숨겨진 레이어의 수 및 너비와 같이 모델 선택에 영향을 미치는 모델 하이퍼파라미터

2. SGD(Stochastic Gradient Descent)의 학습률 및 KNN(k Nearest Neighbors) 분류자의 최근접 이웃 수와 같은 학습 알고리즘의 속도와 품질에 영향을 주는 알고리즘 하이퍼파라미터
이 튜토리얼에서는 Keras Tuner를 사용하여 이미지 분류 애플리케이션에 하이퍼튜닝을 수행합니다.

## 설정


In [ ]:
import tensorflow as tf
from tensorflow import keras

import IPython


Keras Tuner를 설치하고 가져옴


In [ ]:
!pip install -q -U keras-tuner
import kerastuner as kt

## 데이터세트 다운로드 및 준비하기

이 튜토리얼에서는 keras tuner를 사용하여 fashion mnist 데이터세트에서 의류 이미지를 분류하는 머신러닝 모델에 가장 적합한 하이퍼파라미터를 찾습니다.  

데이터를 로드

In [ ]:
(img_train,label_train),(img_test,label_test) = keras.datasets.fashion_mnist.load_data()

In [ ]:
#Normalize pixel values between 0 and 1
img_train = img_train.astype('float32') / 255.0
img_test = img_test.astype('float32') / 255.0

## 모델 정의하기  
하이퍼튜닝을 위한 모델을 빌드할 때는 모델 아키텍처와 더불어 하이퍼파라미터 검색 공간도 정의. 하이퍼튜닝을 위해 설정하는 모델을 *하이퍼 모델*이라고 한다.  
두 가지 접근 방식을 통해 하이퍼 모델을 정의할 수 있다.  
- 모델 빌더 함수 사용
- Keras Tuner API의 ```HyperModel ``` 클래스를 하위 클래스 화

또한 두개의 사전 정의된 ```HyperModel ``` - 클래스인 HyperXception과 HyperResNet을 컴퓨터 비전 애플리케이션에 사용할 수 있다.  

이 튜토리얼에서는 모델 빌더 함수를 사용하여 이미지 분류 모델을 정의한다. 모델 빌더 함수는 컴파일된 모델을 반환하고 인라인으로 정희한 하이퍼파라미터를 사용하여 모델을 하이퍼 튜닝한다.


In [ ]:
def model_builder(hp):
    model = keras.Sequential()
    model.add(keras.layers.Flatten(input_shape=(28,28)))

    #Tune the number of units in the first Dense layer
    #Choose an optimal value between 32-512

    hp_units = hp.Int('units',min_value = 32, max_value = 512 ,step =32)
    model.add(keras.layers.Dense(units = hp_units, activation ='relu'))
    model.add(keras.layers.Dense(10))

    #Tune the leaning rate for the optimizer
    #Choose an optimal value form 0.01, 0.001, or 0.0001
    hp_learning_rate = hp.Choice('learning_rate',values=[1e-2,1e-3,1e-4])

    model.compile(optimizer = keras.optimizers.Adam(learning_rate=hp_learning_rate),
                  loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics = ['accuracy'])
    return model


In [ ]:
tuner = kt.Hyperband(model_builder,
                     objective = 'val_accuracy',
                     max_epochs=10,
                     factor = 3,
                     directory = 'my_dir',
                     project_name='intro_to_kt')

In [ ]:
class ClearTrainingOutput(tf.keras.callbacks.Callback):
    def on_train_end(*arg,**kwargs):
        IPython.display.clear_output(wait=True)

In [ ]:
tuner.search(img_train,label_train,epochs=10,validation_data=(img_test,label_test),callbacks=[ClearTrainingOutput()])

#Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials = 1)[0]
print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")


In [ ]:
# Build the model with the optimal hyperparameters and train it on the data
model = tuner.hypermodel.build(best_hps)
model.fit(img_train, label_train, epochs = 10, validation_data = (img_test, label_test))